In [1]:
import numpy as np
import csv
import pandas as pd
import re
import os
import json
import math

### file path
root_path = 'assets/case0-npc_cut_in/'

In [18]:
def npc_from_txt(filepath):
    f = open(filepath, 'r')
    fframe = f.readlines()
    
    #append timestamp
    df_ts = pd.DataFrame(columns=['npc_ts'])
    #append npc
    df_lc = pd.DataFrame(columns=['npc_x','npc_y','theta'])
    
    for row in fframe:
        if row.find('timestamp_sec') != -1:
            ts = re.findall("(?<=[AZaz])?[0-9.+-]+",row) #exclude "timestamp_sec:"
            ts = float(ts[0]) #convert to data format
            l_no = fframe.index(row)
            
            lc_x = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no + 11])
            lc_x = float(lc_x[0])
            lc_y = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no + 12])
            lc_y = float(lc_y[0])
            theta = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no +15])
            theta = float(theta[0])
            
            d_0 = {'npc_ts':[ts]}
            d_df0 = pd.DataFrame(data=d_0)
            df_ts = pd.concat([df_ts, d_df0])

            d_1 = {'npc_x':[lc_x], 'npc_y':[lc_y],'theta':[theta]}
            d_df1 = pd.DataFrame(data=d_1)
            df_lc = pd.concat([df_lc, d_df1])
    df = pd.concat([df_ts, df_lc], axis=1)

    return df

In [19]:
import pandas as pd
import re

def npc_from_txt_wid(filepath):
    with open(filepath, 'r') as f:
        content = f.read()
    
    blocks = re.split(r'module_name: "perception_obstacle"', content)

    df = pd.DataFrame()
    
    for block in blocks:
        data = dict()
        ts_match = re.search(r'timestamp_sec: ([\d.+-]+)', block)
        if ts_match:
            ts = float(ts_match.group(1))   
            data['npc_ts'] = ts
        subblocks = re.split(r'perception_obstacle \{', block)
        for i, subblock in enumerate(subblocks):
            id_match = re.search(r'id: (\d+)', subblock)
            x_match = re.search(r'position \{\s+x: ([\d.+-]+)', subblock)
            y_match = re.search(r'y: ([\d.+-]+)', subblock)
            theta_match = re.search(r'theta: ([\d.+-]+)', subblock)
            
            if id_match and x_match and y_match and theta_match:
                npc_id = int(id_match.group(1))
                x = float(x_match.group(1))
                y = float(y_match.group(1))
                theta = float(theta_match.group(1))

                data.update({
                    f'NPC{npc_id}X': x,
                    f'NPC{npc_id}Y': y,
                    f'NPC{npc_id}Theta': theta,
                })

        if data:
            df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
 
    return df


In [20]:
def ego_from_txt(filepath):
    f = open(filepath, 'r')
    fframe = f.readlines()
    
    #append timestamp
    df_ts = pd.DataFrame(columns=['ego_ts'])
    #append position
    df_lc = pd.DataFrame(columns=['egoX','egoY','heading'])
    
    for row in fframe:
        if row.find('timestamp_sec') != -1:
            ts = re.findall("(?<=[AZaz])?[0-9.+-]+",row) #exclude "timestamp_sec:"
            ts = float(ts[0]) #convert to data format
            l_no = fframe.index(row)
            
            lc_x = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no + 6])
            lc_x = float(lc_x[0])
            lc_y = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no + 7])
            lc_y = float(lc_y[0])
            heading = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no +31])
            heading = float(heading[0])
            
            d_0 = {'ego_ts':[ts]}
            d_df0 = pd.DataFrame(data=d_0)
            df_ts = pd.concat([df_ts, d_df0])

            d_1 = {'egoX':[lc_x], 'egoY':[lc_y],'heading':[heading]}
            d_df1 = pd.DataFrame(data=d_1)
            df_lc = pd.concat([df_lc, d_df1])
    df = pd.concat([df_ts, df_lc], axis=1)

    return df

In [21]:
def cmd_from_txt(filepath):
    f = open(filepath, 'r')
    fframe = f.readlines()
    
    # Initialize an empty DataFrame
    df = pd.DataFrame(columns=['cmd_ts', 'throttle', 'brake', 'steering'])
    
    for l_no, row in enumerate(fframe):
        if re.match(r'^header {', row):
            ts = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 1])
            ts = float(ts[0])

            throttle = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 11])
            if throttle:
                throttle = float(throttle[0])
            else:
                throttle = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 12])
                if throttle:
                    throttle = float(throttle[0])
                else:
                    print(f"Error in {filepath}, line {l_no}: Throttle value not found")
                    print(f"timestamp = {ts}")

            brake = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 12])
            brake = float(brake[0])

            steering = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 14])
            steering = float(steering[0])

            col_df = pd.concat([pd.DataFrame({'cmd_ts': [ts]}), 
                                 pd.DataFrame({'throttle': [throttle]}), 
                                 pd.DataFrame({'brake': [brake]}), 
                                 pd.DataFrame({'steering': [steering]})], axis=1)
            
            df = pd.concat([df, col_df], ignore_index=True)

    return df

In [22]:
def align_and_average(serial):
    data_path = root_path
    df_npc = npc_from_txt_wid(data_path+f'txt_record/{serial}_obstacle.txt')
    df_ego = ego_from_txt(data_path+    f'txt_record/{serial}_pose.txt')
    df_cmd = cmd_from_txt(data_path+    f'txt_record/{serial}_control.txt')

    delta_ts = df_ego['ego_ts'].iloc[0] - df_npc['npc_ts'].iloc[0]
    df_npc['npc_ts'] += delta_ts

    aligned_ego_list = []
    avg_cmd_list = []
    aligned_npc_list = []
    
    last_npc_idx = 0
    
    for i in range(df_npc.shape[0]):
        ego_indices = np.where(df_ego['ego_ts'] < df_npc['npc_ts'].iloc[i])[0]
        if ego_indices.size == 0:
            continue
        ego_idx = ego_indices[-1]
        
        cmd_indices = np.where((df_cmd['cmd_ts'] >= df_npc['npc_ts'].iloc[last_npc_idx]) & 
                               (df_cmd['cmd_ts'] < df_npc['npc_ts'].iloc[i]))[0]

        if cmd_indices.size > 0:
            aligned_ego_list.append(df_ego.iloc[ego_idx])
            avg_cmd_list.append(df_cmd.iloc[cmd_indices].mean())
            aligned_npc_list.append(df_npc.iloc[i])
        
        last_npc_idx = i

    df_aligned_ego = pd.DataFrame(aligned_ego_list).reset_index(drop=True)
    df_avg_cmd = pd.DataFrame(avg_cmd_list).reset_index(drop=True)
    df_aligned_npc = pd.DataFrame(aligned_npc_list).reset_index(drop=True)

    return df_aligned_ego, df_aligned_npc, df_avg_cmd

In [23]:
# # uncomment for testing
# aligned_ego, aligned_npc, avg_cmd = align_and_average(16)

# # convert numpy to pandas DataFrame
# df_ego = pd.DataFrame(aligned_ego) 
# df_cmd = pd.DataFrame(avg_cmd)     
# df_npc = pd.DataFrame(aligned_npc)        

# # save as csv file
# df_ego.to_csv(root_path+'aligned_ego.csv', index=False)
# df_cmd.to_csv(root_path+'avg_cmd.csv', index=False)
# df_npc.to_csv(root_path+'aligned_npc.csv', index=False)

In [24]:
def process_serials(serials):
    merge_df = pd.DataFrame()
    
    for i in range(serials):
        print(f"Processing serial {i}")
        aligned_ego, aligned_npc, avg_cmd = align_and_average(i)
        tmp = aligned_ego['ego_ts'].to_frame(name='time')
        tmp['id'] = i + 1  
        tmp = pd.concat([tmp, aligned_ego, aligned_npc, avg_cmd], axis=1)
        
        columns_to_drop = ['ego_ts', 'cmd_ts', 'npc_ts']
        for col in columns_to_drop:
            if col not in tmp.columns:
                print(f"Warning: Column '{col}' not found in DataFrame at iteration {i}. Available columns: {tmp.columns}")
            else:
                tmp = tmp.drop(columns=col)
        merge_df = pd.concat([merge_df, tmp], ignore_index=True)
    return merge_df


In [25]:
# to extract static feature
import pandas as pd
import re

def parse_simulation_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    data = []
    iteration_data = {}
    
    for line in lines:
        if "Iteration" in line:
            if iteration_data:
                data.append(iteration_data)
                iteration_data = {}
            iteration_data['id'] = int(re.search(r'\d+', line).group())
        elif "Weather" in line:
            weather_data = re.findall(r'(\w+)=([\d.]+)', line)
            for item in weather_data:
                iteration_data[item[0]] = float(item[1])
        elif "Time of day" in line:
            iteration_data['daytime'] = float(re.search(r'[\d.]+', line).group())
        elif "Collision" in line:
            match = re.search(r"Collision: (True|False)", line)
            if match:
                value = match.group(1)
                iteration_data['result'] = 1 if value == "True" else 0
    
    if iteration_data:
        data.append(iteration_data)
    
    df = pd.DataFrame(data)
    
    return df

file_path = root_path+'simulation_data.txt'
df = parse_simulation_data(file_path)

# # uncomment for testing
# print(df)
df.to_csv(root_path+'features.csv')

In [26]:
### edit # serials to proceed
df_merge = pd.DataFrame()
df_merge = process_serials(31)
df_merge = df_merge.reindex(columns=['id', 'time'] + [col for col in df_merge.columns if col not in ['id', 'time']])
df_merge.insert(0, "", range(1, len(df_merge) + 1))
df_merge.to_csv(root_path+'data_processed.csv',index=False)

Processing serial 0
Processing serial 1
Processing serial 2
Processing serial 3
Processing serial 4
Processing serial 5
Processing serial 6
Processing serial 7
Processing serial 8
Processing serial 9
Processing serial 10
Processing serial 11
Processing serial 12
Processing serial 13
Processing serial 14
Processing serial 15
Processing serial 16
Processing serial 17
Processing serial 18
Processing serial 19
Processing serial 20
Processing serial 21
Processing serial 22
Processing serial 23
Processing serial 24
Processing serial 25
Processing serial 26
Processing serial 27
Processing serial 28
Processing serial 29
Processing serial 30


In [28]:
def compute_perpendicular_distance(x, y, line):
    x, y = float(x), float(y)  # Convert x and y to float
    min_distance = float('inf')
    sign = 1  # Positive for left, negative for right
    
    for i in range(len(line) - 1):
        x1, y1 = float(line[i]['x']), float(line[i]['y'])  # Convert string to float
        x2, y2 = float(line[i+1]['x']), float(line[i+1]['y'])  # Convert string to float
        
        v_length = math.sqrt((x2-x1)**2 + (y2-y1)**2)
        dot_product = ((x-x1)*(x2-x1) + (y-y1)*(y2-y1)) / v_length
        
        if dot_product < 0:
            distance = math.sqrt((x-x1)**2 + (y-y1)**2)
        elif dot_product > v_length:
            distance = math.sqrt((x-x2)**2 + (y-y2)**2)
        else:
            # Compute the cross product to determine the side
            cross_product = (x - x1) * (y2 - y1) - (y - y1) * (x2 - x1)
            if cross_product < 0:
                sign = -1
            distance = abs((y2-y1)*x - (x2-x1)*y + x2*y1 - y2*x1) / v_length
        
        # Update the minimum distance with the correct sign
        if distance < min_distance:
            min_distance = distance * sign
            
    return min_distance


def find_lane_and_distance(x, y, lanes):
    if x and y:
        x, y = float(x), float(y)  # Convert x and y to float
    else:
        return None, None
    min_distance = float('inf')
    closest_lane_id = None
    
    for lane_id, lane_data in lanes.items():
        central_curve = lane_data['central_curve']
        distance = compute_perpendicular_distance(x, y, central_curve)
        
        if abs(distance) < abs(min_distance):
            min_distance = distance
            closest_lane_id = lane_id
          
    # If the nearest lane distance is less than 2m, return that lane and distance, otherwise return None
    if abs(min_distance) < 2:
        lane_number = re.findall(r'\d+', closest_lane_id)
        return lane_number[0] if lane_number else closest_lane_id, min_distance
    else:
        return None, None
    
# # uncomment for testing
# with open('assets/lane_info.json', 'r') as f:
#      lanes = json.load(f)
# x, y = 592418.56176757812, 4134988.5171813965  # Example point
# lane_id, distance = find_lane_and_distance(x, y, lanes)

# if lane_id:
#     print(f"The point ({x}, {y}) is on lane {lane_id} with a distance of {distance} to the central curve.")
# else:
#     print(lane_id, distance)



df = pd.read_csv(root_path+'data_processed.csv')
# Load lanes data from JSON
with open('assets/lane_info.json', 'r') as f:
    lanes = json.load(f)

for index, row in df.iterrows():
    lane_id, distance = find_lane_and_distance(row['egoX'], row['egoY'],lanes)
    df.at[index, 'egoLane'] = lane_id
    df.at[index, 'egoLaneOffset'] = distance
    
    lane_id, distance = find_lane_and_distance(row['NPC1X'], row['NPC1Y'],lanes)
    df.at[index, 'NPC1Lane'] = lane_id
    df.at[index, 'NPC1LaneOffset'] = distance

    # lane_id, distance = find_lane_and_distance(row['NPC2X'], row['NPC2Y'],lanes)
    # df.at[index, 'NPC2Lane'] = lane_id
    # df.at[index, 'NPC2LaneOffset'] = distance

df.to_csv(root_path+'data_processed.csv', index=False)
